# Measures of Spread: Understanding Variability

## Introduction

In the previous notebook, we learned about **central tendency** - finding the "typical" or "center" value of data. But knowing the center isn't enough!

Consider two fields:
- **Field A**: Yields over 5 years: [4.0, 4.1, 4.0, 4.1, 4.0] tons/hectare → Mean = 4.04
- **Field B**: Yields over 5 years: [2.0, 5.0, 3.0, 6.0, 4.2] tons/hectare → Mean = 4.04

Both fields have the **same mean**, but they tell very different stories!
- Field A is **consistent** and **predictable** (low variability)
- Field B is **volatile** and **risky** (high variability)

This is where **measures of spread** become essential.

### What You'll Learn

1. ✅ Calculate **range** and **IQR** (interquartile range)
2. ✅ Understand **variance** ⭐ (CRITICAL for PCA!)
3. ✅ Calculate **standard deviation** ⭐ (CRITICAL for PCA!)
4. ✅ Use **coefficient of variation** to compare variability
5. ✅ Visualize spread with box plots and distribution plots
6. ✅ **Connect to PCA**: Understand why PCA maximizes variance

### Why This is CRITICAL for Machine Learning

🎯 **Principal Component Analysis (PCA) finds directions of MAXIMUM VARIANCE**

Understanding what variance means and why it matters is **absolutely essential** for understanding PCA!

Let's dive in! 🌾📊

In [ ]:
# Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
np.random.seed(42)

print("✓ Setup complete!")

---

## 1. Range and Interquartile Range (IQR)

### Range

The **range** is the simplest measure of spread:

$$\text{Range} = \text{Maximum} - \text{Minimum}$$

✅ **Advantages**: Very easy to calculate and understand
⚠️ **Disadvantages**: Only uses 2 values, sensitive to outliers

### Interquartile Range (IQR)

The **IQR** measures the spread of the middle 50% of data:

$$\text{IQR} = Q_3 - Q_1$$

Where:
- $Q_1$ = First quartile (25th percentile)
- $Q_3$ = Third quartile (75th percentile)

✅ **Advantages**: Robust to outliers, shows spread of central data

### Agricultural Example: Temperature Variability

In [ ]:
# Daily high temperatures (°C) for two different months
march_temps = np.array([15, 17, 16, 18, 19, 16, 17, 18, 20, 17, 16, 18, 19, 21, 18])
july_temps = np.array([28, 30, 29, 31, 33, 30, 29, 31, 30, 32, 29, 30, 31, 30, 32])

print("March temperatures (°C):")
print(march_temps)
print(f"Range: {np.ptp(march_temps)}°C  (ptp = peak-to-peak = max-min)")
print(f"IQR: {np.percentile(march_temps, 75) - np.percentile(march_temps, 25):.1f}°C")
print()

print("July temperatures (°C):")
print(july_temps)
print(f"Range: {np.ptp(july_temps)}°C")
print(f"IQR: {np.percentile(july_temps, 75) - np.percentile(july_temps, 25):.1f}°C")
print()

print("💡 Interpretation:")
print("   March has wider range → More variable temperatures")
print("   IQR confirms March is less consistent")

In [ ]:
# Box plot visualization
fig, ax = plt.subplots(figsize=(12, 6))

data_temps = [march_temps, july_temps]
bp = ax.boxplot(data_temps, labels=['March', 'July'], patch_artist=True,
                boxprops=dict(facecolor='lightblue', alpha=0.7),
                medianprops=dict(color='red', linewidth=2.5),
                whiskerprops=dict(linewidth=1.5),
                capprops=dict(linewidth=1.5))

# Color boxes differently
bp['boxes'][0].set_facecolor('lightcoral')
bp['boxes'][1].set_facecolor('lightyellow')

# Annotations
ax.annotate('IQR', xy=(1, np.percentile(march_temps, 25)), 
           xytext=(0.7, 17),
           arrowprops=dict(arrowstyle='<->', color='blue', lw=2),
           fontsize=13, fontweight='bold', color='blue')

ax.annotate('Range', xy=(2, np.max(july_temps)), 
           xytext=(2.3, 31),
           arrowprops=dict(arrowstyle='<->', color='red', lw=2),
           fontsize=13, fontweight='bold', color='red')

ax.set_ylabel('Temperature (°C)', fontsize=12, fontweight='bold')
ax.set_title('Temperature Variability: March vs July\n(Box shows IQR, Whiskers show Range)', 
            fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\n📊 Box Plot Anatomy:")
print("   - Box bottom = Q1 (25th percentile)")
print("   - Red line = Median (Q2, 50th percentile)")
print("   - Box top = Q3 (75th percentile)")
print("   - Box height = IQR (Q3 - Q1)")
print("   - Whiskers = Range (min to max)")

---

## 2. Variance ⭐ (CRITICAL for PCA!)

**Variance** is THE most important measure of spread for machine learning!

### Mathematical Definition

**Population variance**:
$$\sigma^2 = \frac{1}{N} \sum_{i=1}^{N} (x_i - \mu)^2$$

**Sample variance** (what we usually calculate):
$$s^2 = \frac{1}{n-1} \sum_{i=1}^{n} (x_i - \bar{x})^2$$

Where:
- $n$ = sample size
- $x_i$ = individual values
- $\bar{x}$ = sample mean
- $n-1$ = degrees of freedom (Bessel's correction)

### Intuition: Average Squared Distance from Mean

Variance measures **how far** each data point is from the mean, **on average**.

**Step-by-step process:**
1. Calculate the mean $\bar{x}$
2. For each point, find deviation: $(x_i - \bar{x})$
3. **Square** each deviation: $(x_i - \bar{x})^2$  (makes all values positive!)
4. Average the squared deviations

### Why We Square the Deviations

If we just averaged $(x_i - \bar{x})$ without squaring:
- Positive deviations would cancel negative deviations
- The sum would always be zero!
- We'd learn nothing about spread

Squaring ensures:
✅ All deviations are positive
✅ Larger deviations get more weight (squared)
✅ We get a meaningful measure of spread

### Agricultural Example: Yield Variability

In [ ]:
# Two fields: same mean, different variance
field_A = np.array([4.0, 4.1, 4.0, 4.1, 4.0, 4.05, 4.0, 4.05])  # Low variance
field_B = np.array([2.0, 5.0, 3.0, 6.0, 4.2, 5.5, 3.3, 3.2])  # High variance

# Both have approximately same mean!
mean_A = np.mean(field_A)
mean_B = np.mean(field_B)

print("Field A (consistent yields):")
print(field_A)
print(f"Mean: {mean_A:.3f} tons/hectare")
print()

print("Field B (variable yields):")
print(field_B)
print(f"Mean: {mean_B:.3f} tons/hectare")
print()
print("✓ Both fields have approximately the SAME mean!")
print("  But they're very different...")

In [ ]:
# Calculate variance step-by-step for Field A
print("=== Calculating Variance for Field A (Step-by-Step) ===")
print()

# Step 1: Calculate mean
mean_A = np.mean(field_A)
print(f"Step 1: Mean = {mean_A:.3f}")
print()

# Step 2: Calculate deviations
deviations_A = field_A - mean_A
print("Step 2: Deviations from mean:")
for i, (val, dev) in enumerate(zip(field_A, deviations_A)):
    print(f"  {val:.2f} - {mean_A:.3f} = {dev:+.3f}")
print(f"  Sum of deviations: {np.sum(deviations_A):.10f} (≈ 0, always!)")
print()

# Step 3: Square the deviations
squared_devs_A = deviations_A ** 2
print("Step 3: Squared deviations:")
for i, (dev, sq_dev) in enumerate(zip(deviations_A, squared_devs_A)):
    print(f"  ({dev:+.3f})² = {sq_dev:.6f}")
print()

# Step 4: Average (with n-1 for sample variance)
sum_sq_devs = np.sum(squared_devs_A)
n = len(field_A)
variance_A_manual = sum_sq_devs / (n - 1)

print(f"Step 4: Variance = Sum(squared deviations) / (n-1)")
print(f"                 = {sum_sq_devs:.6f} / {n-1}")
print(f"                 = {variance_A_manual:.6f}")
print()

# Verify with NumPy
variance_A_numpy = np.var(field_A, ddof=1)  # ddof=1 for sample variance
print(f"NumPy verification: {variance_A_numpy:.6f}")
print(f"✓ Match: {np.isclose(variance_A_manual, variance_A_numpy)}")

In [ ]:
# Compare variances of both fields
var_A = np.var(field_A, ddof=1)
var_B = np.var(field_B, ddof=1)

print("\n=== Variance Comparison ===")
print(f"Field A variance: {var_A:.4f} (tons/hectare)²")
print(f"Field B variance: {var_B:.4f} (tons/hectare)²")
print(f"\nField B has {var_B/var_A:.1f}x MORE variance than Field A!")
print()
print("💡 Interpretation:")
print("   - Higher variance = More spread out = More VARIABILITY")
print("   - Lower variance = More clustered = More CONSISTENT")
print("   - Field B is much riskier for farming!")

In [ ]:
# Visualization: Variance as spread
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Raw data comparison
ax1 = axes[0, 0]
x_A = range(len(field_A))
x_B = range(len(field_B))
ax1.plot(x_A, field_A, 'o-', color='blue', markersize=10, linewidth=2, label='Field A (low variance)')
ax1.plot(x_B, field_B, 's-', color='red', markersize=10, linewidth=2, label='Field B (high variance)')
ax1.axhline(mean_A, color='blue', linestyle='--', linewidth=1.5, alpha=0.7)
ax1.axhline(mean_B, color='red', linestyle='--', linewidth=1.5, alpha=0.7)
ax1.set_xlabel('Year', fontsize=11, fontweight='bold')
ax1.set_ylabel('Yield (tons/hectare)', fontsize=11, fontweight='bold')
ax1.set_title('Yield Over Time: Same Mean, Different Variance', fontsize=13, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Plot 2: Deviations from mean (Field A)
ax2 = axes[0, 1]
ax2.bar(x_A, field_A - mean_A, color='lightblue', edgecolor='blue', linewidth=2)
ax2.axhline(0, color='black', linewidth=2)
ax2.set_xlabel('Year', fontsize=11, fontweight='bold')
ax2.set_ylabel('Deviation from Mean', fontsize=11, fontweight='bold')
ax2.set_title(f'Field A: Deviations (variance={var_A:.4f})', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)

# Plot 3: Deviations from mean (Field B)
ax3 = axes[1, 0]
ax3.bar(x_B, field_B - mean_B, color='lightcoral', edgecolor='red', linewidth=2)
ax3.axhline(0, color='black', linewidth=2)
ax3.set_xlabel('Year', fontsize=11, fontweight='bold')
ax3.set_ylabel('Deviation from Mean', fontsize=11, fontweight='bold')
ax3.set_title(f'Field B: Deviations (variance={var_B:.4f})', fontsize=13, fontweight='bold')
ax3.grid(True, alpha=0.3)

# Plot 4: Histograms showing spread
ax4 = axes[1, 1]
ax4.hist(field_A, bins=8, alpha=0.5, color='blue', edgecolor='blue', label=f'Field A (var={var_A:.4f})')
ax4.hist(field_B, bins=8, alpha=0.5, color='red', edgecolor='red', label=f'Field B (var={var_B:.4f})')
ax4.axvline(mean_A, color='blue', linestyle='--', linewidth=2)
ax4.axvline(mean_B, color='red', linestyle='--', linewidth=2)
ax4.set_xlabel('Yield (tons/hectare)', fontsize=11, fontweight='bold')
ax4.set_ylabel('Frequency', fontsize=11, fontweight='bold')
ax4.set_title('Distribution Comparison', fontsize=13, fontweight='bold')
ax4.legend(fontsize=10)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Key Observations:")
print("   - Top-left: Field B fluctuates much more")
print("   - Middle row: Deviations are much larger in Field B")
print("   - Bottom-right: Field B data is more SPREAD OUT")
print("\n💡 This spread is exactly what VARIANCE measures!")

### Why Variance Matters for PCA

🎯 **Key Insight**: **Variance measures how much information is in your data!**

Think about it:
- **Low variance** → Data points are similar → Less information → Less useful for prediction
- **High variance** → Data points are diverse → More information → More useful!

**PCA's goal**: Find directions in your data with **MAXIMUM VARIANCE**

Why? Because directions with high variance:
- Capture the most information
- Best represent data differences
- Are most useful for analysis and prediction

**You'll see this formula in PCA:**
```python
# PCA finds principal components that maximize:
variance_along_component = np.var(X_projected)
```

Now you understand why!

---

## 3. Standard Deviation ⭐

**Standard deviation (SD)** is simply the **square root of variance**:

$$\sigma = \sqrt{\sigma^2} = \sqrt{\frac{1}{N} \sum_{i=1}^{N} (x_i - \mu)^2}$$

For samples:
$$s = \sqrt{s^2} = \sqrt{\frac{1}{n-1} \sum_{i=1}^{n} (x_i - \bar{x})^2}$$

### Why Take the Square Root?

Variance has **squared units**: $(\text{tons/hectare})^2$

Standard deviation brings us back to **original units**: $\text{tons/hectare}$

This makes it:
- ✅ Easier to interpret
- ✅ Comparable to the mean
- ✅ More intuitive for reporting

### Interpretation: "Typical" Distance from Mean

SD tells you the **typical deviation** you'd expect from the mean.

### The 68-95-99.7 Rule (Empirical Rule)

For **approximately normal** distributions:
- **68%** of data falls within **1 standard deviation** of the mean
- **95%** of data falls within **2 standard deviations** of the mean
- **99.7%** of data falls within **3 standard deviations** of the mean

In [ ]:
# Agricultural example: Soil nitrogen measurements
nitrogen_levels = np.array([42, 48, 51, 45, 50, 49, 47, 52, 46, 48, 50, 49, 51, 47, 48,
                           50, 49, 48, 51, 47, 49, 50, 48, 46, 52])

mean_N = np.mean(nitrogen_levels)
var_N = np.var(nitrogen_levels, ddof=1)
std_N = np.std(nitrogen_levels, ddof=1)

print("Soil Nitrogen Levels (ppm) across 25 fields:")
print(nitrogen_levels)
print()
print(f"Mean:     {mean_N:.2f} ppm")
print(f"Variance: {var_N:.4f} (ppm)²  ← Squared units!")
print(f"Std Dev:  {std_N:.2f} ppm      ← Original units!")
print()
print(f"💡 Interpretation: Nitrogen levels typically vary by ±{std_N:.2f} ppm from the mean of {mean_N:.2f} ppm")

# Verify relationship
print(f"\n✓ Verification: √variance = {np.sqrt(var_N):.2f} = std_dev = {std_N:.2f}")

In [ ]:
# Visualization: 68-95-99.7 Rule
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Histogram with SD bands
ax1.hist(nitrogen_levels, bins=12, alpha=0.7, color='forestgreen', edgecolor='darkgreen', density=True)
ax1.axvline(mean_N, color='red', linestyle='--', linewidth=2.5, label=f'Mean = {mean_N:.1f} ppm')
ax1.axvline(mean_N - std_N, color='orange', linestyle='--', linewidth=2, label=f'±1 SD')
ax1.axvline(mean_N + std_N, color='orange', linestyle='--', linewidth=2)
ax1.axvline(mean_N - 2*std_N, color='blue', linestyle=':', linewidth=2, label=f'±2 SD')
ax1.axvline(mean_N + 2*std_N, color='blue', linestyle=':', linewidth=2)

# Add shaded regions
ax1.axvspan(mean_N - std_N, mean_N + std_N, alpha=0.2, color='orange', label='68% of data')
ax1.axvspan(mean_N - 2*std_N, mean_N + 2*std_N, alpha=0.1, color='blue', label='95% of data')

ax1.set_xlabel('Nitrogen Level (ppm)', fontsize=11, fontweight='bold')
ax1.set_ylabel('Density', fontsize=11, fontweight='bold')
ax1.set_title('Standard Deviation: Typical Distance from Mean\n(68-95-99.7 Rule)', fontsize=13, fontweight='bold')
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)

# Plot 2: Normal distribution showing empirical rule
x = np.linspace(mean_N - 4*std_N, mean_N + 4*std_N, 1000)
y = stats.norm.pdf(x, mean_N, std_N)
ax2.plot(x, y, 'b-', linewidth=2.5, label='Normal Distribution')
ax2.fill_between(x, 0, y, where=(x >= mean_N - std_N) & (x <= mean_N + std_N), 
                 alpha=0.3, color='orange', label='68% (±1σ)')
ax2.fill_between(x, 0, y, where=(x >= mean_N - 2*std_N) & (x <= mean_N + 2*std_N), 
                 alpha=0.2, color='blue', label='95% (±2σ)')
ax2.axvline(mean_N, color='red', linestyle='--', linewidth=2)

ax2.set_xlabel('Nitrogen Level (ppm)', fontsize=11, fontweight='bold')
ax2.set_ylabel('Probability Density', fontsize=11, fontweight='bold')
ax2.set_title('Empirical Rule for Normal Distribution', fontsize=13, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Count data in each region
within_1sd = np.sum((nitrogen_levels >= mean_N - std_N) & (nitrogen_levels <= mean_N + std_N))
within_2sd = np.sum((nitrogen_levels >= mean_N - 2*std_N) & (nitrogen_levels <= mean_N + 2*std_N))

print(f"\n📊 Actual data distribution:")
print(f"   Within ±1 SD: {within_1sd}/{len(nitrogen_levels)} = {within_1sd/len(nitrogen_levels)*100:.1f}% (expect ~68%)")
print(f"   Within ±2 SD: {within_2sd}/{len(nitrogen_levels)} = {within_2sd/len(nitrogen_levels)*100:.1f}% (expect ~95%)")

---

## 4. Coefficient of Variation (CV)

The **coefficient of variation** is the **relative** variability:

$$\text{CV} = \frac{\sigma}{\mu} \times 100\%$$

or for samples:

$$\text{CV} = \frac{s}{\bar{x}} \times 100\%$$

### Why Use CV?

CV allows you to compare variability of datasets with **different units or scales**.

**Example**: Which is more variable?
- Soil pH: Mean = 6.5, SD = 0.5
- Nitrogen: Mean = 50 ppm, SD = 10 ppm

You can't directly compare SD values (different units!), but you CAN compare CV values.

In [ ]:
# Compare variability of different soil properties
np.random.seed(42)

# Generate data with different scales
pH = np.random.normal(6.5, 0.5, 30)
nitrogen = np.random.normal(50, 10, 30)
phosphorus = np.random.normal(25, 8, 30)
organic_matter = np.random.normal(3.5, 1.2, 30)

# Calculate statistics
properties = {
    'pH': pH,
    'Nitrogen (ppm)': nitrogen,
    'Phosphorus (ppm)': phosphorus,
    'Organic Matter (%)': organic_matter
}

print("=== Variability Comparison of Soil Properties ===")
print()
print(f"{'Property':<20} {'Mean':<10} {'SD':<10} {'CV (%)':<10}")
print("-" * 50)

cv_values = {}
for prop_name, prop_data in properties.items():
    mean = np.mean(prop_data)
    sd = np.std(prop_data, ddof=1)
    cv = (sd / mean) * 100
    cv_values[prop_name] = cv
    print(f"{prop_name:<20} {mean:>9.2f} {sd:>9.2f} {cv:>9.2f}")

print()
most_variable = max(cv_values, key=cv_values.get)
least_variable = min(cv_values, key=cv_values.get)

print(f"💡 Interpretation:")
print(f"   Most variable:  {most_variable} (CV = {cv_values[most_variable]:.1f}%)")
print(f"   Least variable: {least_variable} (CV = {cv_values[least_variable]:.1f}%)")
print()
print("   CV allows comparison despite different units and scales!")

In [ ]:
# Visualization: CV comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Standard deviations (can't compare directly!)
prop_names = list(properties.keys())
sds = [np.std(properties[p], ddof=1) for p in prop_names]
ax1.bar(range(len(prop_names)), sds, color='steelblue', edgecolor='black', linewidth=1.5)
ax1.set_xticks(range(len(prop_names)))
ax1.set_xticklabels(prop_names, rotation=45, ha='right')
ax1.set_ylabel('Standard Deviation', fontsize=11, fontweight='bold')
ax1.set_title('Standard Deviations\n⚠️ Can\'t compare - different units!', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')
for i, (name, sd) in enumerate(zip(prop_names, sds)):
    ax1.text(i, sd + 0.3, f'{sd:.1f}', ha='center', fontweight='bold')

# Plot 2: Coefficients of variation (can compare!)
cvs = [cv_values[p] for p in prop_names]
colors = ['gold' if cv == max(cvs) else 'lightcoral' if cv == min(cvs) else 'steelblue' 
          for cv in cvs]
ax2.bar(range(len(prop_names)), cvs, color=colors, edgecolor='black', linewidth=1.5)
ax2.set_xticks(range(len(prop_names)))
ax2.set_xticklabels(prop_names, rotation=45, ha='right')
ax2.set_ylabel('Coefficient of Variation (%)', fontsize=11, fontweight='bold')
ax2.set_title('Coefficients of Variation\n✓ Comparable - unitless percentages!', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')
for i, (name, cv) in enumerate(zip(prop_names, cvs)):
    ax2.text(i, cv + 1, f'{cv:.1f}%', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 Key Point: CV enables fair comparison across different scales!")

---

## Key Takeaways

### 💡 Main Concepts

1. **Range**: Max - Min (simple but sensitive to outliers)
2. **IQR**: Q3 - Q1 (robust measure of middle 50% spread)
3. **Variance** ⭐⭐: Average squared deviation from mean
   - **Measures information content**
   - **CRITICAL for PCA** - PCA maximizes variance!
   - Units are squared
4. **Standard Deviation** ⭐: Square root of variance
   - Same units as original data
   - "Typical" distance from mean
   - Easier to interpret than variance
5. **Coefficient of Variation**: Relative variability (SD/Mean × 100%)
   - Enables comparison across different scales
   - Unitless percentage

### 🔗 Connection to PCA

**Why variance is CRITICAL for PCA:**

1. **Variance = Information**: High variance directions contain more information
2. **PCA's Goal**: Find directions of MAXIMUM variance
3. **Principal Components**: Ordered by variance (PC1 has highest variance)
4. **Data Reduction**: Keep components with high variance, discard low variance

**You'll see in PCA module:**
```python
# PCA finds these directions:
explained_variance = [var(PC1), var(PC2), ...]
# PC1 has max variance, PC2 has 2nd max, etc.
```

Now you understand what variance means and why PCA maximizes it!

### 📊 Practical Guidelines

**When analyzing agricultural data:**

- **Use variance/SD** to quantify:
  - Yield consistency across fields or years
  - Weather variability (risk assessment)
  - Soil property uniformity

- **Use CV** to compare:
  - Different soil properties (pH vs nitrogen)
  - Different crops or regions
  - Relative risk/stability

- **Use IQR** when:
  - Data has outliers
  - Need robust spread measure
  - Creating box plots

### 🌾 Agricultural Decision-Making

**Low variance (consistent)**:
- ✅ Predictable yields
- ✅ Lower risk
- ✅ Easier planning
- Example: Irrigated field with controlled conditions

**High variance (variable)**:
- ⚠️ Unpredictable yields
- ⚠️ Higher risk
- ⚠️ Requires contingency planning
- Example: Rainfed field with weather dependence

---

## Next Steps

Excellent work! You now understand:
- ✅ How to measure data spread
- ✅ Why variance matters for ML
- ✅ Why PCA maximizes variance

**Continue to the next notebook:**
`03_covariance_correlation.ipynb` - **MOST CRITICAL FOR PCA!**

In the next notebook, you'll learn:
- **Covariance**: How variables vary TOGETHER
- **Covariance Matrix**: The input to PCA!
- **Correlation**: Standardized covariance

This is the **MOST IMPORTANT** notebook for understanding PCA, because:
🎯 **PCA decomposes the covariance matrix to find principal components!**

You're building the perfect foundation! 🚀